In [124]:
%load_ext autoreload
%autoreload 2
# add . to module name
import sys
sys.path.append('../src/')

In [1]:
import folium

In [2]:
cologne_center = ( 50.9384, 6.96)
bbox = {
    "max_lat": 50.938361 + 0.05,
    "min_lat": 50.938361 - 0.05,
    "max_lon": 6.959974 + 0.04,
    "min_lon": 6.959974 - 0.07
}
print(bbox)

{'max_lat': 50.988361, 'min_lat': 50.888361, 'max_lon': 6.999974, 'min_lon': 6.889974}


In [3]:
m = folium.Map(location=cologne_center, zoom_start=12)

folium.Rectangle(
    bounds=[[bbox["min_lat"], bbox["min_lon"]], [bbox["max_lat"], bbox["max_lon"]]],
    color='red',
    fill=True,
    fill_color='red'
).add_to(m)



m

In [5]:
m = folium.Map(location=cologne_center, zoom_start=12)
m.add_child(folium.LatLngPopup())
m

In [ ]:
locations = [
	()
]

In [115]:
import overpy

def order_ways_and_nodes(result):
    ways_dict = {way.id: [node for node in way.nodes] for way in result.ways}

    current_way_id, current_way_nodes = ways_dict.popitem()
    ordered_nodes = current_way_nodes

    while ways_dict:
        for next_way_id, next_way_nodes in ways_dict.items():
            if ordered_nodes[-1] == next_way_nodes[0]:
                ordered_nodes.extend(next_way_nodes[1:])
                current_way_id = next_way_id
                break
            elif ordered_nodes[-1] == next_way_nodes[-1]:
                ordered_nodes.extend(reversed(next_way_nodes[:-1]))
                current_way_id = next_way_id
                break
        ways_dict.pop(current_way_id)

    return [(node.lat, node.lon) for node in ordered_nodes]

def fetch_boundary_coords(city_name):
    api = overpy.Overpass()

    query = f"""
    [out:json][timeout:50];
    area["name"="{city_name}"]->.searchArea;
    relation["boundary"="administrative"]["admin_level"="6"](area.searchArea);
    out body;
    >;
    out skel qt;
    """

    result = api.query(query)
    boundary_coords = order_ways_and_nodes(result)

    return boundary_coords

boundary_coords = fetch_boundary_coords("Köln")


In [199]:
from shapely.geometry import Polygon
polygon = Polygon([(lon, lat) for lat, lon in boundary_coords])

In [217]:
from package.geometa import GeoMeta
geometa = GeoMeta(boundary=polygon)
geometa.save("../data/geometa.pkl")

In [218]:
from package import storage
stops_df = storage.read_df("../data/cleaned/stops.csv")

In [219]:
m = folium.Map(location=geometa.get_center_lat_lon(), zoom_start=11)

for _, stop in stops_df.iterrows():
	folium.CircleMarker(
		location=[stop["stop_lat"], stop["stop_lon"]],
		radius=2,
		color="black",
		fill=True,
		fill_color="black"
	).add_to(m)

geometa.add_to_folium_map(m)